In [1]:
# import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

# Model architecture

In [3]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("openai-community/gpt2")
print(config)

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.57.3",
  "use_cache": true,
  "vocab_size": 50257
}



In [35]:
input_text = "Hello, I am a language model and I"
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(input_text, return_tensors="pt")
print(inputs)
# token=tokenizer.tokenize(input_text)
# print("Tokens:", token)
# input_ids = tokenizer.convert_tokens_to_ids(token)
# print("IDs List:", input_ids)
# import torch
# input_ids = torch.tensor([input_ids])
# print("Input IDs Tensor:", input_ids)

{'input_ids': tensor([[15496,    11,   314,   716,   257,  3303,  2746,   290,   314]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [37]:
generated_ids = model.generate(
    **inputs,
    max_length=100,
    num_beams=5,  # بررسی ۵ مسیر موازی برای یافتن بهترین جمله
    no_repeat_ngram_size=2,  # جلوگیری از تکرار دو کلمه پشت سر هم
    early_stopping=True,  # توقف به محض رسیدن به پایان منطقی
    pad_token_id=tokenizer.eos_token_id
)

result = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(result)

Hello, I am a language model and I have a lot of work to do.

I have been working on this project for a long time and it has been a pleasure to work with you. I would like to thank you for taking the time to answer my questions and to give me your feedback. If you have any questions or comments, please feel free to contact me.


In [38]:
# ذخیره مدل و توکنایزر در یک پوشه محلی
model.save_pretrained("./my_gpt2_model")
tokenizer.save_pretrained("./my_gpt2_model")

('./my_gpt2_model\\tokenizer_config.json',
 './my_gpt2_model\\special_tokens_map.json',
 './my_gpt2_model\\vocab.json',
 './my_gpt2_model\\merges.txt',
 './my_gpt2_model\\added_tokens.json',
 './my_gpt2_model\\tokenizer.json')

# Fine tuning

In [39]:
from peft import LoraConfig, get_peft_model

# ۱. تنظیمات LoRA
config = LoraConfig(
    r=8,  # رتبه ماتریس‌ها (هر چه بیشتر باشد، ظرفیت یادگیری بیشتر و حافظه بیشتر مصرف می‌شود)
    lora_alpha=32,  # ضریب مقیاس‌گذاری
    target_modules=["c_attn"],  # لایه‌های هدف در GPT-2 (معمولاً لایه‌های Attention)
    lora_dropout=0.1,
    task_type="CAUSAL_LM",
)

# ۲. اعمال تنظیمات روی مدل اصلی
model = get_peft_model(model, config)

# ۳. چاپ پارامترهای قابل آموزش
model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364


C:\Users\shahrooz\PycharmProjects\JupyterProject2\.venv\Lib\site-packages\peft\tuners\lora\layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [40]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# # ۱. آماده‌سازی دیتاست
# def load_dataset(file_path, tokenizer):
#     return TextDataset(
#         tokenizer=tokenizer,
#         file_path=file_path,
#         block_size=128 # طول هر قطعه متن برای آموزش
#     )
#
# train_dataset = load_dataset("my_data.txt", tokenizer)
#
# ۲. جمع‌آوری‌کننده داده (Data Collator)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from datasets import load_dataset

# دانلود یک دیتاست آماده (مثلاً جملات کوتاه انگلیسی)
train_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-2-raw-v1/train-00000-of-00001.p(…):   0%|          | 0.00/6.36M [00:00<?, ?B/s]

wikitext-2-raw-v1/validation-00000-of-00(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./gpt2-lora",
        per_device_train_batch_size=4,
        learning_rate=3e-4,  # در LoRA معمولاً نرخ یادگیری را کمی بالاتر می‌برند
        num_train_epochs=1,
        logging_steps=10,
    ),
    train_dataset=train_dataset,  # همان دیتاستی که قبلاً ساختیم
    data_collator=data_collator,
)

trainer.train()

In [ ]:
model.save_pretrained("./gpt2-lora-adapter")

# Inference

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# الف. بارگذاری مدل اصلی (Base Model)
base_model_name = "openai-community/gpt2"
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# ب. بارگذاری و سوار کردن آداپتور روی مدل اصلی
model = PeftModel.from_pretrained(base_model, "./gpt2-lora-adapter")

# حالا مدل آماده تولید متن با دانش جدید است
inputs = tokenizer("Your prompt here", return_tensors="pt")
outputs = model.generate(**inputs, max_length=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))